the missing value is on AFRM , it's for the next quarter it's not labeled +1q that's the reason behind that.

In [1]:
import time as _tm
import requests as _rq
import pandas as _pd
import openpyxl
API_TOKEN = "67ffece4b2ae08.94077168"
BASE_URL  = "https://eodhd.com/api"
HEADERS   = {"User-Agent": "EPS-Trend/1.1"}
PAUSE_SEC = 1.1                     


def _get_json(url: str) -> dict:
    r = _rq.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()


def _pull_trend(ticker: str) -> dict:

    url = (
        f"{BASE_URL}/fundamentals/{ticker}"
        f"?filter=Earnings::Trend"           
        f"&api_token={API_TOKEN}&fmt=json"
    )
    data = _get_json(url)

    if "Earnings" in data:          
        return data.get("Earnings", {}).get("Trend", {})
    return data                     


def _latest_snapshots(trend: dict) -> dict[str, dict]:
    latest = {}
    for date_str, rec in trend.items():
        p = rec.get("period")
        if not p:
            continue
        ts = _pd.to_datetime(date_str, errors="coerce")
        if p not in latest or ts > latest[p]["_ts"]:
            rec["_ts"] = ts
            latest[p] = rec
    return latest


def _row_for(ticker: str) -> dict:
    trend = _pull_trend(ticker)
    if not trend:
        return {"Ticker": ticker.split('.', 1)[0], "ERROR": "no Trend"}

    snap = _latest_snapshots(trend)

    periods = {
        "Curr Qtr": "0q",
        "Next Qtr": "+1q",
        "Curr Yr":  "0y",
        "Next Yr":  "+1y",
    }

    def val(rec, fld):
        return float(rec.get(fld)) if rec and rec.get(fld) not in (None, "") else None

    row = {"Ticker": ticker.split('.', 1)[0]}
    for label, code in periods.items():
        rec = snap.get(code, {})
        row[f"{label} Curr"] = val(rec, "epsTrendCurrent")
        row[f"{label} 7"]    = val(rec, "epsTrend7daysAgo")
        row[f"{label} 30"]   = val(rec, "epsTrend30daysAgo")
        row[f"{label} 60"]   = val(rec, "epsTrend60daysAgo")
        row[f"{label} 90"]   = val(rec, "epsTrend90daysAgo")
    return row


def build_table(tickers: list[str]) -> _pd.DataFrame:
    rows = []
    for tk in tickers:
        print(f"{tk} ...", end="", flush=True)
        try:
            rows.append(_row_for(tk))
            print("ok")
        except Exception as e:
            print(f"ERR: {e}")
            rows.append({"Ticker": tk.split('.', 1)[0], "ERROR": str(e)})
        _tm.sleep(PAUSE_SEC)
    return _pd.DataFrame(rows)


def save(df: _pd.DataFrame, fname="eps_trend.csv"):
    if fname.lower().endswith(".xlsx"):
        df.to_excel(fname, index=False, engine="openpyxl")
    else:
        df.to_csv(fname, index=False, encoding="utf-8")
    print(f"\n✅  Saved {len(df)} rows → {fname}")


if __name__ == "__main__":
    raw1 = ['aapl', 'spot', 'afrm', 'rddt' , 'nflx', 'uber', 'u', 'msft', 'amd']
    raw = [
        "NET", "CRS", "ORLY", "DOCN", "TEM", "VST", "CVS", "TER", "RIVN", "NTGR",
        "SPOT", "UBER", "SNOW", "ACHR", "ASAN", "RKLB", "DPZ", "LYFT", "NEGG", "MSFT",
        "AAPL", "ARM", "AMZN", "META", "NFLX", "NVDA", "GOOG", "RDFN", "MDB", "Z",
        "INTC", "AMD", "AFRM", "COIN", "TSLA", "HOOD", "CAVA", "F", "GM", "SMR", "U",
        "RDDT", "SHOP", "SOFI"
    ]
    tickers = [f"{t}.US" for t in raw1]
    tbl1 = build_table(tickers)
    save(tbl1, "eps_trend.xlsx")


aapl.US ...ok
spot.US ...ok
afrm.US ...ok
rddt.US ...ok
nflx.US ...ok
uber.US ...ok
u.US ...ok
msft.US ...ok
amd.US ...ok


NameError: name 'tbl' is not defined

In [2]:
display(tbl1)

,Ticker,Curr Qtr Curr,Curr Qtr 7,Curr Qtr 30,Curr Qtr 60,Curr Qtr 90,Next Qtr Curr,Next Qtr 7,Next Qtr 30,Next Qtr 60,...,Curr Yr Curr,Curr Yr 7,Curr Yr 30,Curr Yr 60,Curr Yr 90,Next Yr Curr,Next Yr 7,Next Yr 30,Next Yr 60,Next Yr 90
0,aapl,1.6118,1.6052,1.6067,1.6080,1.6571,1.4639,1.4646,1.4929,1.4971,...,7.2452,7.2449,7.3154,7.3157,7.3628,7.9652,7.9792,8.1812,8.1961,8.2352
1,spot,2.2124,2.4360,2.4735,2.4441,2.0747,2.4830,2.7341,2.7923,2.7872,...,8.9465,10.4897,10.5938,10.6516,9.3406,12.9334,13.3379,13.5464,13.3665,11.5606
2,afrm,0.3156,0.3120,-0.0450,-0.0900,-0.1733,NaN,NaN,NaN,NaN,...,1.5810,-0.0300,-0.0900,-0.0900,-1.2500,1.8476,0.7933,0.6100,-0.3433,-0.2025
3,rddt,0.1795,0.1232,0.1346,0.1399,0.0742,0.3206,0.3259,0.3457,0.3416,...,1.2203,1.0633,1.1399,1.1367,0.8427,2.2550,2.1288,2.2684,2.2941,1.8752
4,nflx,7.0537,7.0565,6.2550,6.2472,6.2508,6.6268,6.6202,6.9938,6.9753,...,25.5752,25.5509,24.7614,24.7207,24.7451,30.7776,30.7266,30.2709,30.2507,30.2257
5,uber,0.5029,0.5019,0.5056,0.5054,0.4608,0.5773,0.5777,0.5918,0.5915,...,2.4264,2.4250,2.4427,2.4414,2.3319,3.3105,3.3141,3.3461,3.3449,3.2665
6,u,-0.3665,-0.3672,-0.3680,-0.3691,-0.3668,-0.2973,-0.2973,-0.2973,-0.2955,...,-1.1295,-1.1323,-1.1373,-1.1283,-1.2057,-0.9040,-0.9103,-0.9147,-0.8858,-1.0166
7,msft,3.2160,3.2147,3.2134,3.2129,3.1535,3.5526,3.5476,3.5476,3.5453,...,13.4089,13.1393,13.1552,13.1543,13.1545,15.1163,15.0006,15.0400,15.0648,15.0781
8,amd,0.9336,0.9412,0.9348,0.9378,0.9439,0.8815,0.9003,1.0077,1.0167,...,4.4021,4.4130,4.6523,4.6933,4.9128,5.9452,6.0025,6.2184,6.3106,6.8178
